
# Binary trees in Haskell

![A tree](https://farm6.staticflickr.com/5134/5519064502_b2055733b5_z.jpg)

This is a simple implementation of binary trees (and some of their basic functions) using [Haskell](https://www.haskell.org/). 

## Definition

Short and clean: a tree is either empty or composed by a node that has two trees (left and right) under it:

In [11]:
data Tree a = EmptyTree | Node a (Tree a) (Tree a) deriving (Show, Read, Eq)  


## Node insertion

Given a node with value $x$, anything under it to the left should be less than $x$ and to the right should be bigger than $x$:

In [12]:
insertNode :: (Ord a) => a -> Tree a -> Tree a
insertNode x EmptyTree = Node x EmptyTree EmptyTree
insertNode x (Node y left right) 
    | x == y = Node y left right
    | x < y  = Node y (insertNode x left) right
    | x > y  = Node y left (insertNode x right)

## Building trees

The following function uses insertion to build binary trees from lists:

In [13]:
buildTree = foldl (flip insertNode) EmptyTree 

### Example

In [14]:
t = buildTree [2, 1, 3, 4, 2.5, 1.5, 0.5]

In [15]:
t

Node 2.0 (Node 1.0 (Node 0.5 EmptyTree EmptyTree) (Node 1.5 EmptyTree EmptyTree)) (Node 3.0 (Node 2.5 EmptyTree EmptyTree) (Node 4.0 EmptyTree EmptyTree))

## Extreme values

The minimum value in a tree is obtained by starting at the root and follow the left branch as deeply as possible:

In [16]:
minimumInTree :: (Ord a) => Tree a -> a
minimumInTree EmptyTree = error "Tree is empty"
minimumInTree (Node x left right) 
    | left == EmptyTree = x
    | otherwise = minimumInTree left

### Example:

In [19]:
minimumInTree t

0.5

## Node deletion

Nodes are easy to delete if they have at most one branch. Just push everything up. If they have both branches, the value of the node needs to be replaced by that of the value in which is the successor in the tree considering only the nodes below. It sounds tricky but it is not that hard:

In [20]:
deleteNode :: (Ord a, Eq a) => a -> Tree a -> Tree a
deleteNode x EmptyTree = EmptyTree
deleteNode x (Node y left right)
    | x < y = Node y (deleteNode x left) right
    | x > y = Node y left (deleteNode x right)
    | x == y && left == EmptyTree = right
    | x == y && right == EmptyTree = left
    | otherwise = Node m left (deleteNode m right)
    where m = minimumInTree right

### Examples

If the element is not in the tree, nothing should happen:

In [49]:
deleteNode 17 t

Node 2.0 (Node 1.0 (Node 0.5 EmptyTree EmptyTree) (Node 1.5 EmptyTree EmptyTree)) (Node 3.0 (Node 2.5 EmptyTree EmptyTree) (Node 4.0 EmptyTree EmptyTree))

Let's corroborate it:

In [50]:
deleteNode 17 t == t

True

Now we can try to delete a node that is actually in the tree:

In [26]:
deleteNode 4 t

Node 2.0 (Node 1.0 (Node 0.5 EmptyTree EmptyTree) (Node 1.5 EmptyTree EmptyTree)) (Node 3.0 (Node 2.5 EmptyTree EmptyTree) EmptyTree)

But 4 is easy: it is at the bottom of the tree (doesn't have any branches to deal with!).

What about the root? This one is a little harder:

In [27]:
deleteNode 2 t

Node 2.5 (Node 1.0 (Node 0.5 EmptyTree EmptyTree) (Node 1.5 EmptyTree EmptyTree)) (Node 3.0 EmptyTree (Node 4.0 EmptyTree EmptyTree))

## Finding values in a tree

This is one of the main reasons binary trees are useful: finding elements (once the tree has been built) is pretty —$O(\log n)$— fast.

In [33]:
findInTree :: (Ord a) => a -> Tree a -> Bool
findInTree x EmptyTree = False
findInTree x (Node y left right)
    | x == y = True
    | x < y = findInTree x left
    | x > y = findInTree x right

### Examples

In [34]:
findInTree 2 t

True

In [53]:
findInTree 17 t

False

This gives us another way of testing our deleting function:

In [36]:
findInTree 2 $ deleteNode 2 t

False

## Ordering a tree

It is also easy (given the tree) to generate a sorted list with its elements:

In [54]:
orderTree :: (Ord a) => Tree a -> [a]
orderTree EmptyTree = []
orderTree (Node x left right) = (orderTree left) ++ [x] ++ (orderTree right)

### Example

In [38]:
orderTree t

[0.5,1.0,1.5,2.0,2.5,3.0,4.0]

## Reflecting a tree

I've read that this is a common problem in technical interviews: write a function such that outputs a mirror image of a given tree (this of course reverses the order rule described above).

In this setting the function is almost trivial:

In [39]:
mirrorTree :: (Ord a) => Tree a -> Tree a
mirrorTree EmptyTree = EmptyTree
mirrorTree (Node x left right) = Node x (mirrorTree right) (mirrorTree left)

### Example

In [46]:
mirrorTree t

Node 2.0 (Node 3.0 (Node 4.0 EmptyTree EmptyTree) (Node 2.5 EmptyTree EmptyTree)) (Node 1.0 (Node 1.5 EmptyTree EmptyTree) (Node 0.5 EmptyTree EmptyTree))

And of course the reflection of the reflection gives you back the original tree:

In [44]:
mirrorTree (mirrorTree t) == t 

True